In [1]:
import os
import json
import time
import shutil

license = ""

if license and "json" in license:

    with open(license, "r") as creds_in:
        creds = json.loads(creds_in.read())

        for key in creds.keys():
            os.environ[key] = creds[key]
else:
    raise Exception("License JSON File is not specified")

In [ ]:
!pip install --upgrade -q https://pypi.johnsnowlabs.com/$SPARK_OCR_SECRET/spark-ocr/spark_ocr-6.0.0-py3-none-any.whl

!pip install --upgrade -q https://pypi.johnsnowlabs.com/$SECRET/spark-nlp-jsl/spark_nlp_jsl-6.0.0-py3-none-any.whl

!pip install -q spark-nlp==$PUBLIC_VERSION

!pip install -q pandas

!pip install -q matplotlib

In [ ]:
## RESTART SESSION!!!

In [2]:
from sparkocr import start
import os
import json
import time
import shutil

license = ""

if license and "json" in license:

    with open(license, "r") as creds_in:
        creds = json.loads(creds_in.read())

        for key in creds.keys():
            os.environ[key] = creds[key]
else:
    raise Exception("License JSON File is not specified")

In [ ]:
extra_configurations = {
    "spark.extraListeners": "com.johnsnowlabs.license.LicenseLifeCycleManager", #required
    "spark.sql.legacy.allowUntypedScalaUDF" : "true", #required
    "spark.executor.instances" : "7", #change as per system
    "spark.executor.cores" : "16", #change as per system
    "spark.executor.memory" : "130G", #change as per system
    "spark.driver.memory" : "100G", #change as per system
    "spark.sql.shuffle.partitions" : "896" #change as per system
}

# Not needed for Google Collab
os.environ['JAVA_HOME'] = '/home/linuxbrew/.linuxbrew/Cellar/openjdk@17/17.0.15'

spark = start(secret=os.environ.get("SPARK_OCR_SECRET"),
              nlp_secret=os.environ.get("SECRET"),
              nlp_internal=True,
              nlp_jsl=True,
              nlp_version=os.environ.get("PUBLIC_VERSION"),
              extra_conf=extra_configurations)

spark

In [6]:
import numpy as np
import pandas as pd
import os

#Pyspark Imports
from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel
from pyspark.sql import functions as F

# Necessary imports from Spark OCR library
import sparkocr
from sparkocr import start
from sparkocr.transformers import *
from sparkocr.enums import *
from sparkocr.utils import *

# import sparknlp packages
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp_jsl
from sparknlp_jsl.annotator import *
from collections import Counter
from sparknlp.pretrained import PretrainedPipeline

In [7]:
def evaluate_predictions(SOURCE_GT_PATH, DF_SAVE_PATH, SAVE_MAPPING_PATH):
    """
    Method to Calculate Precision, Recall and F1-Score
    Saves final file with prediction, ground truth, precision, recall
    """
    
    def calculate_metrics(preds, gts):
      gt_counter = Counter(gts)
      pred_counter = Counter(preds)

      tp = 0
      for item in pred_counter:
          if item in gt_counter:
              tp += min(pred_counter[item], gt_counter[item])

      fp = sum(pred_counter.values()) - tp
      fn = sum(gt_counter.values()) - tp

      precision = tp / (tp + fp) if (tp + fp) else 0
      recall = tp / (tp + fn) if (tp + fn) else 0

      return precision, recall

    with open(SOURCE_GT_PATH, "r") as f:
        ground_truth = json.load(f)

    df_predictions = spark.read.format("parquet").load(DF_SAVE_PATH)

    predictions_by_file = {}

    for row in df_predictions.select("path").distinct().toLocalIterator():
        file_path = row.asDict()["path"]
        filename = os.path.basename(file_path)

        if filename not in ground_truth:
            continue

        extracted_results = []
        rows = df_predictions.filter(F.col("path") == file_path).select("positions_ner")

        for r in rows.toLocalIterator():
            for ner in r.asDict()["positions_ner"]:
                extracted_results.append(ner.asDict()["result"])

        predictions_by_file[filename] = extracted_results

    summary = {}
    all_precisions = []
    all_recalls = []

    for filename, predictions in predictions_by_file.items():
        gt_values = ground_truth[filename]
        gt_values = [i.replace("-year-old", "") for i in gt_values]
        predictions = [i.replace("-year-old", "") for i in predictions]
        precision, recall = calculate_metrics([i.replace(" ", "") for i in predictions], 
                                              [i.replace(" ", "") for i in gt_values])

        all_precisions.append(precision)
        all_recalls.append(recall)

        summary[filename] = {
            "precision": round(precision, 4),
            "recall": round(recall, 4),
            "gt": gt_values,
            "pred": predictions
        }

        print(f"Filename: {filename} | Precision: {precision:.4f} | Recall: {recall:.4f}")

    avg_precision = round(sum(all_precisions) / len(all_precisions), 4)
    avg_recall = round(sum(all_recalls) / len(all_recalls), 4)
    f1_score = round(2 * (avg_precision * avg_recall) / (avg_precision + avg_recall), 4)

    print(f"\nOverall Precision: {avg_precision}")
    print(f"Overall Recall: {avg_recall}")
    print(f"F1 Score: {f1_score}")

    with open(SAVE_MAPPING_PATH, "w") as f:
        json.dump(summary, f, indent=4)

    print(f"Mapping File Saved To : {SAVE_MAPPING_PATH}")

In [ ]:
pdf_to_image = PdfToImage() \
  .setInputCol("content") \
  .setSplitNumBatch(10) \
  .setOutputCol("image_raw") \
  .setImageType(ImageType.TYPE_3BYTE_BGR) \
  .setSplittingStategy(SplittingStrategy.FIXED_NUMBER_OF_PARTITIONS)

ocr = ImageToText() \
    .setInputCol("image_raw") \
    .setOutputCol("text") \
    .setIgnoreResolution(False) \
    .setPageIteratorLevel(PageIteratorLevel.SYMBOL) \
    .setPageSegMode(PageSegmentationMode.SPARSE_TEXT) \
    .setWithSpaces(True) \
    .setKeepLayout(False) \
    .setConfidenceThreshold(70)

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document") \
    .setCleanupMode("shrink_full") 

abbreviations = ['Bros', 'No', 'al', 'vs', 'etc', 'Fig', 'Dr', 'Prof', 'PhD', 'MD', 'Co', 'Corp', 'Inc', 'bros', 'VS', 'Vs', 'ETC', 'fig', 'dr', 'prof', 'PHD', 'phd', 'md', 'co', 'corp', 'inc', 'Jan', 'Feb', 'Mar', 'Apr', 'Jul', 'Aug', 'Sep', 'Sept', 'Oct', 'Nov', 'Dec', 'St', 'st', 'AM', 'PM', 'am', 'pm', 'e.g', 'f.e', 'i.e']
sentence_detector = SentenceDetectorDLModel.pretrained("sentence_detector_dl", "en") \
    .setInputCols(["document"]) \
    .setOutputCol("sentence") \
    .setImpossiblePenultimates(abbreviations) \
    .setUseCustomBoundsOnly(False) \
    .setSplitLength(2147483647) \
    .setExplodeSentences(False)

tokenizer = Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("token") \
    .setContextChars(['.', ',', ';', ':', '!', '?', '*', '"', "'"])

zero_shot_ner = ZeroShotNerModel.pretrained("zero_shot_ner_roberta", "en", "clinical/models")\
    .setEntityDefinitions(
        {
            "NAME" : ["What is the Patient Name?"],
            "DATE" : ["What is the Patient Date of Birth?", "What is the DOB?"],
            "IDNUM" : ["What is the SSN Number?", "What is the SSN?"]
        })\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("zero_shot_ner")\
    .setPredictionThreshold(0.1)

ner_converter = NerConverterInternal()\
    .setInputCols(["sentence", "token", "zero_shot_ner"])\
    .setOutputCol("ner_chunk") \
    .setThreshold(0.60)

deid_obfuscated = DeIdentification() \
    .setInputCols(["sentence", "token", "ner_chunk"]) \
    .setOutputCol("obfuscated") \
    .setMode("obfuscate") \
    .setKeepMonth(True) \
    .setKeepYear(True) \
    .setObfuscateDate(True) \
    .setSameEntityThreshold(0.7) \
    .setKeepTextSizeForObfuscation(True) \
    .setFakerLengthOffset(2) \
    .setReturnEntityMappings(True) \
    .setDays(2) \
    .setMappingsColumn("aux") \
    .setIgnoreRegex(True) \
    .setGroupByCol("path") \
    .setRegion("us") \
    .setSeed(40) \
    .setConsistentObfuscation(True) \
    .setChunkMatching({"NAME" : 0.60, "DATE" : 0.60, "IDNUM" : 0.60})

cleaner = NerOutputCleaner() \
    .setInputCol("aux") \
    .setOutputCol("new_aux") \
    .setOutputNerCol("positions_ner")

position_finder = PositionFinder() \
    .setInputCols("positions_ner") \
    .setOutputCol("coordinates") \
    .setPageMatrixCol("positions")

draw_regions = ImageDrawRegions() \
  .setInputCol("image_raw") \
  .setInputRegionsCol("coordinates") \
  .setRectColor(Color.black) \
  .setFilledRect(True) \
  .setOutputCol("image_with_regions")

stages = [
    pdf_to_image,
    ocr,
    document_assembler,
    sentence_detector,
    tokenizer,
    zero_shot_ner,
    ner_converter,
    deid_obfuscated,
    cleaner,
    position_finder,
    draw_regions
]

pipe = Pipeline(stages=stages)

In [9]:
SOURCE_PDF_PATH = "/workspace/PDF_FILES_MEDIUM/*"
DF_SAVE_PATH = "/workspace/zero_shot/"
SOURCE_GT_PATH = "/workspace/pdf_deid_zero_shot_gts.json"
SAVE_MAPPING_PATH = "/workspace/zero_shot_result_mapping.json"
SAVE_OUTPUT_PDF = "/workspace/zero_shot_pdf_output/"

os.makedirs(SAVE_OUTPUT_PDF, exist_ok=True)

In [10]:
df = spark.read.format("binaryFile").load(SOURCE_PDF_PATH)
result = pipe.fit(df).transform(df)
result.write.format('parquet').mode('overwrite').save(DF_SAVE_PATH)

25/05/14 09:31:51 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/05/14 09:35:41 WARN DAGScheduler: Broadcasting large task binary with size 1255.3 KiB


In [11]:
evaluate_predictions(SOURCE_GT_PATH=SOURCE_GT_PATH, 
                     DF_SAVE_PATH=DF_SAVE_PATH, 
                     SAVE_MAPPING_PATH=SAVE_MAPPING_PATH)

Filename: PDF_Deid_Deidentification_Medium_9.pdf | Precision: 0.9375 | Recall: 1.0000
Filename: PDF_Deid_Deidentification_Medium_7.pdf | Precision: 1.0000 | Recall: 1.0000
Filename: PDF_Deid_Deidentification_Medium_8.pdf | Precision: 1.0000 | Recall: 1.0000
Filename: PDF_Deid_Deidentification_Medium_6.pdf | Precision: 1.0000 | Recall: 1.0000
Filename: PDF_Deid_Deidentification_Medium_2.pdf | Precision: 0.9375 | Recall: 1.0000
Filename: PDF_Deid_Deidentification_Medium_4.pdf | Precision: 1.0000 | Recall: 1.0000
Filename: PDF_Deid_Deidentification_Medium_0.pdf | Precision: 1.0000 | Recall: 1.0000
Filename: PDF_Deid_Deidentification_Medium_1.pdf | Precision: 1.0000 | Recall: 1.0000
Filename: PDF_Deid_Deidentification_Medium_3.pdf | Precision: 1.0000 | Recall: 1.0000

Overall Precision: 0.9861
Overall Recall: 1.0
F1 Score: 0.993
Mapping File Saved To : /workspace/zero_shot_result_mapping.json


In [12]:
OBFUSCATED_IMAGE_COL = "image_with_regions"

img_to_pdf = ImageToPdf() \
    .setPageNumCol("pagenum") \
    .setOriginCol("path") \
    .setOutputCol("pdf") \
    .setInputCol(OBFUSCATED_IMAGE_COL) \
    .setAggregatePages(True)

source = spark.read.format("parquet").load(DF_SAVE_PATH)
result_pdf = img_to_pdf.transform(source)

for row in result_pdf.select("path", "pdf").toLocalIterator():
  filename = row.asDict()["path"]
  basename = os.path.basename(filename)

  savename = os.path.join(SAVE_OUTPUT_PDF, basename)
    
  pdfFile = open(savename, "wb")
  pdfFile.write(row.asDict()["pdf"])
  pdfFile.close()